## Install Libraries and Packages

In [1]:
%pip install --quiet --upgrade \
    pip \
    python-dotenv \
    datasets \
    accelerate \
    peft \
    bitsandbytes \
    transformers \
    trl \
    sentencepiece

Note: you may need to restart the kernel to use updated packages.


## Import Necessary Packages

In [2]:
import os
import torch

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
    TrainingArguments,
    logging,
)

from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training

from trl import ORPOConfig, ORPOTrainer, setup_chat_format

## Load Model and Tokenizer

In [3]:
model_path = "./gpt2_tagalog"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [4]:
# Set the pad token to be the same as the eos token
tokenizer.pad_token = tokenizer.eos_token

## Prepare and analyze the data for finetune

In [5]:
import pandas as pd

tagalog_keyboard_data = pd.read_csv("/teamspace/studios/this_studio/tagalog-keyboard-data.csv")
tagalog_keyboard_data.head()

,Incomplete Sentence,Suggestions,Rejected Suggestions
0,Kumain ako ng ma,"['manok', 'mangga', 'mais']","['mataas', 'marahas', 'marunong']"
1,Maglalaro tayo ng b,"['bola', 'basketball', 'badminton']","['bahay', 'bata', 'buwan']"
2,Ang ganda ng a,"['araw', 'awit', 'arte']","['alis', 'aso', 'anak']"
3,Pumunta kami sa ba,"['bakuran', 'bayan', 'bahay']","['bagyo', 'baril', 'bakasyon']"
4,Maglalaro tayo ng b,"['basketball', 'bola', 'badminton']","['buwan', 'bahay', 'bata']"


In [6]:
# Rename the columns
tagalog_keyboard_data = tagalog_keyboard_data.rename(columns={
    'Incomplete Sentence': 'prompt',
    'Suggestions': 'chosen',
    'Rejected Suggestions': 'rejected'
})
tagalog_keyboard_data.head()

,prompt,chosen,rejected
0,Kumain ako ng ma,"['manok', 'mangga', 'mais']","['mataas', 'marahas', 'marunong']"
1,Maglalaro tayo ng b,"['bola', 'basketball', 'badminton']","['bahay', 'bata', 'buwan']"
2,Ang ganda ng a,"['araw', 'awit', 'arte']","['alis', 'aso', 'anak']"
3,Pumunta kami sa ba,"['bakuran', 'bayan', 'bahay']","['bagyo', 'baril', 'bakasyon']"
4,Maglalaro tayo ng b,"['basketball', 'bola', 'badminton']","['buwan', 'bahay', 'bata']"


In [7]:
from datasets import Dataset
# Create a Hugging Face Dataset from the DataFrame
dataset = Dataset.from_pandas(tagalog_keyboard_data[['chosen', 'rejected', 'prompt']])
dataset

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 100000
})

In [8]:
#Filter the dataset to 1000 examples
dataset = dataset.shuffle(seed=42).select(range(1000))
dataset

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 1000
})

In [9]:
dataset[0]

{'chosen': "['basketball', 'bola', 'badminton']",
 'rejected': "['buwan', 'bata', 'bahay']",
 'prompt': 'Maglalaro tayo ng b'}

In [10]:
import ast

def format_chat_template(row):
    # Parse the Chosen and Rejected columns which are string representations of lists
    chosen_list = ast.literal_eval(row["chosen"])  # Convert string to list of words
    rejected_list = ast.literal_eval(row["rejected"])  # Convert string to list of words

    # Format the "chosen" response as a string with <|user|> and <|assistant|> tokens
    chosen_response = f"<|user|>\n    {row['prompt']}\n<|assistant|>\n    {chosen_list}"

    # Format the "rejected" response as a string with <|user|> and <|assistant|> tokens
    rejected_response = f"<|user|>\n    {row['prompt']}\n<|assistant|>\n    {rejected_list}"

    # Return the row with the same structure as the input, but formatted
    return {
        "chosen": chosen_response,  # Formatted chosen response
        "rejected": rejected_response,  # Formatted rejected response
        "prompt": row["prompt"],  # The prompt remains unchanged
    }

In [11]:
tokenizer.apply_chat_template

<bound method PreTrainedTokenizerBase.apply_chat_template of GPT2TokenizerFast(name_or_path='./gpt2_tagalog', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}>

In [12]:
# Apply the formatting function
formatted_dataset = dataset.map(format_chat_template)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
formatted_dataset[0]

{'chosen': "<|user|>\n    Maglalaro tayo ng b\n<|assistant|>\n    ['basketball', 'bola', 'badminton']",
 'rejected': "<|user|>\n    Maglalaro tayo ng b\n<|assistant|>\n    ['buwan', 'bata', 'bahay']",
 'prompt': 'Maglalaro tayo ng b'}

In [14]:
#Split the data for train and test
formatted_dataset = formatted_dataset.train_test_split(test_size=0.10)
formatted_dataset

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected', 'prompt'],
        num_rows: 900
    })
    test: Dataset({
        features: ['chosen', 'rejected', 'prompt'],
        num_rows: 100
    })
})

In [15]:
def find_linear_layers(model):
    # Initialize a list to store the names of all linear layers
    linear_layers = []
    
    # Iterate over all named modules in the model
    for name, module in model.named_modules():
        # Check if the module is of type Linear (from nn.Module)
        if isinstance(module, torch.nn.Linear):
            # Append the name of the layer to the list
            linear_layers.append(name)
    
    # Return the list of linear layer names
    return linear_layers

# Assuming 'model' is your GPT-2 model
linear_layer_names = find_linear_layers(model)
print(linear_layer_names)

['lm_head']


In [16]:
#default configs
attn_implementation = "eager"
torch_dtype = torch.float16

In [17]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [18]:
orpo_args = ORPOConfig(
    learning_rate=8e-6,
    beta=0.1,
    lr_scheduler_type="linear",
    max_length=1024,
    max_prompt_length=512,
    per_device_train_batch_size=25,
    per_device_eval_batch_size=25,
    gradient_accumulation_steps=1,  # Update after each mini-batch
    optim="paged_adamw_8bit",
    num_train_epochs=10,  # Train for 10 epochs
    eval_strategy="epoch",  # Evaluate after each epoch
    eval_steps=None,  # No need for eval_steps
    logging_steps=1,
    warmup_steps=10,
    save_steps=5,
    output_dir="./tagalog-results-keybaord1",
    save_strategy="epoch",  # Save after each epoch
    load_best_model_at_end=True,  # Load the best model at the end of training
)


In [19]:
#Setup ORPO Trainer
trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=formatted_dataset["train"],
    eval_dataset=formatted_dataset["test"],
    peft_config=peft_config,
    tokenizer=tokenizer,
)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/trl/trainer/orpo_trainer.py:276: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [20]:
#Train the model
trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss,Log Odds Ratio,Log Odds Chosen
1,7.879700,7.779092,0.687600,145.441000,5.818000,-0.794597,-0.795654,0.470000,0.001057,-7.956537,-7.945968,-2.830976,-2.860056,7.709191,-0.699010,0.010599
2,7.547400,7.536638,0.692000,144.509000,5.780000,-0.768216,-0.769629,0.470000,0.001413,-7.696295,-7.682160,-2.811978,-2.841185,7.467031,-0.696067,0.014171
3,7.258800,7.269650,0.696500,143.576000,5.743000,-0.739105,-0.740898,0.470000,0.001794,-7.408984,-7.391046,-2.787481,-2.816983,7.200354,-0.692960,0.017984
4,6.948100,6.997256,0.696500,143.580000,5.743000,-0.709317,-0.711464,0.470000,0.002147,-7.114639,-7.093169,-2.758066,-2.787998,6.928249,-0.690050,0.021529
5,6.787100,6.737466,0.696300,143.615000,5.745000,-0.680880,-0.683327,0.470000,0.002447,-6.833273,-6.808800,-2.726212,-2.756643,6.668709,-0.687570,0.024548
6,6.521800,6.504169,0.697100,143.448000,5.738000,-0.655355,-0.658137,0.470000,0.002782,-6.581373,-6.553554,-2.694555,-2.725513,6.435651,-0.685171,0.027914
7,6.352500,6.308774,0.697100,143.443000,5.738000,-0.633954,-0.637097,0.470000,0.003143,-6.370970,-6.339539,-2.665699,-2.697156,6.240487,-0.682872,0.031549
8,6.077000,6.160684,0.698100,143.237000,5.729000,-0.617769,-0.621246,0.470000,0.003477,-6.212458,-6.177685,-2.642275,-2.674178,6.092593,-0.680902,0.034914
9,5.997100,6.067751,0.700100,142.845000,5.714000,-0.607594,-0.611295,0.490000,0.003701,-6.112950,-6.075936,-2.626698,-2.658900,5.999791,-0.679603,0.037172
10,6.129700,6.035469,0.694600,143.975000,5.759000,-0.604063,-0.607846,0.490000,0.003784,-6.078465,-6.040627,-2.621108,-2.653420,5.967556,-0.679134,0.038002


TrainOutput(global_step=360, training_loss=6.874530659781562, metrics={'train_runtime': 149.7647, 'train_samples_per_second': 60.094, 'train_steps_per_second': 2.404, 'total_flos': 0.0, 'train_loss': 6.874530659781562, 'epoch': 10.0})

In [22]:
# After training
# Step 1: Save the fine-tuned model
trainer.save_model("./tagalog-keyboard-model1")


In [32]:
#Flush the memory
# Flush memory
import gc
#del trainer, model
gc.collect()
torch.cuda.empty_cache()

In [33]:
# Step 2: Load your base GPT-2 model and tokenizer
base_model_path = "./gpt2_tagalog"  # Path to your base GPT-2 Tagalog model
base_model = AutoModelForCausalLM.from_pretrained(base_model_path)
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# Set up the chat format by adding special tokens for user and assistant
base_model, tokenizer = setup_chat_format(base_model, tokenizer)  # Set up for chat format

# Step 3: Load the fine-tuned adapter model (if using PEFT or adapter)
adapter_model_path = "./tagalog-keyboard-model1"  # Path to your fine-tuned model
adapter_model = PeftModel.from_pretrained(base_model, adapter_model_path)  # Use base_model here

# Step 4: Merge the adapter with the base model and unload the adapter
merged_model = adapter_model.merge_and_unload()  # This merges the fine-tuned weights into the base model

# Now, `merged_model` contains the final model (base model + fine-tuned adapter).

# Optionally save the final merged model
merged_model.save_pretrained("./final-tagalog-keyboard-model1")

# You can also save the tokenizer if needed
tokenizer.save_pretrained("./final-tagalog-keyboard-model1")


('./final-tagalog-keyboard-model1/tokenizer_config.json',
 './final-tagalog-keyboard-model1/special_tokens_map.json',
 './final-tagalog-keyboard-model1/vocab.json',
 './final-tagalog-keyboard-model1/merges.txt',
 './final-tagalog-keyboard-model1/added_tokens.json',
 './final-tagalog-keyboard-model1/tokenizer.json')

In [34]:
tokenizer.chat_template

"{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"

In [35]:
# Define the messages (you can have multiple messages in the list)
messages = [
    {'role': 'user', 'content': 'Hello'}
]

# Apply the chat template to the messages
formatted_input = tokenizer.apply_chat_template(messages)
formatted_input

[32000, 320, 282, 199, 30055, 32001, 199]

In [36]:
final_model_path = "./final-tagalog-keyboard-model1"  # Path to the final merged model
model = AutoModelForCausalLM.from_pretrained(final_model_path)  # Load the model
tokenizer = AutoTokenizer.from_pretrained(final_model_path)

In [37]:
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [54]:
# Step 3: Provide a prompt in Tagalog for generating word suggestions
prompt = (
    "Ibinigay ang hindi kumpletong pangungusap, magmungkahi ng mga posibleng salita upang kumpletuhin ito:\n"
    "<|user|>\n    Maglalaro tayo ng b\n<|assistant|>\n    "
    "Mga mungkahi ng sal: "
)

# Step 4: Generate the next word prediction (with multiple suggestions)
result = generator(prompt, max_length=100)  # Adjust max_length as needed

# Print the result
print(result)

[{'generated_text': 'Ibinigay ang hindi kumpletong pangungusap, magmungkahi ng mga posibleng salita upang kumpletuhin ito:\n<|user|>\n    Maglalaro tayo ng b\n<|assistant|>\n    Mga mungkahi ng sal: anayin ang mga salita sa mga salita.|||||||||||||||||||||||||||||||||||||||||'}]
